# 00 - Environment Check

This notebook verifies that all GenAI Vanilla Stack services are accessible from JupyterHub.

## Services Checked
- Ollama (LLM)
- Weaviate (Vector DB)
- Neo4j (Graph DB)
- PostgreSQL (Supabase)
- Redis (Cache)
- ComfyUI (Image Generation)
- n8n (Workflows)
- SearxNG (Search)
- Backend API

In [ ]:
import os
import httpx
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

print("🔍 GenAI Vanilla Stack - Environment Check\n")
print("=" * 60)

## 1. Environment Variables

In [ ]:
services = {
    "Ollama (LLM)": os.getenv("OLLAMA_BASE_URL"),
    "Weaviate (Vector DB)": os.getenv("WEAVIATE_URL"),
    "Neo4j (Graph DB)": os.getenv("NEO4J_URI"),
    "PostgreSQL (Database)": os.getenv("DATABASE_URL", "not set")[:30] + "...",
    "Redis (Cache)": os.getenv("REDIS_URL", "not set")[:30] + "...",
    "ComfyUI (Images)": os.getenv("COMFYUI_BASE_URL"),
    "Supabase (API Gateway)": os.getenv("SUPABASE_URL"),
    "n8n (Workflows)": os.getenv("N8N_BASE_URL"),
    "SearxNG (Search)": os.getenv("SEARXNG_URL"),
    "Backend API": os.getenv("BACKEND_API_URL"),
}

print("\n📋 Environment Variables:")
print("-" * 60)
for name, url in services.items():
    status = "✅" if url else "❌"
    print(f"{status} {name:30s} {url or 'NOT SET'}")

## 2. HTTP Service Connectivity

In [ ]:
async def check_http_service(name, url, endpoint="/"):
    """Check if an HTTP service is reachable."""
    try:
        full_url = f"{url.rstrip('/')}{endpoint}"
        async with httpx.AsyncClient(timeout=5.0) as client:
            response = await client.get(full_url)
            return True, response.status_code
    except Exception as e:
        return False, str(e)

print("\n🌐 HTTP Service Connectivity:")
print("-" * 60)

# Check Ollama
import asyncio

ollama_url = os.getenv("OLLAMA_BASE_URL")
if ollama_url:
    success, result = await check_http_service("Ollama", ollama_url, "/api/tags")
    status = "✅" if success else "❌"
    print(f"{status} Ollama: {result}")

# Check Weaviate
weaviate_url = os.getenv("WEAVIATE_URL")
if weaviate_url:
    success, result = await check_http_service("Weaviate", weaviate_url, "/v1/.well-known/ready")
    status = "✅" if success else "❌"
    print(f"{status} Weaviate: {result}")

# Check ComfyUI
comfyui_url = os.getenv("COMFYUI_BASE_URL")
if comfyui_url:
    success, result = await check_http_service("ComfyUI", comfyui_url, "/system_stats")
    status = "✅" if success else "❌"
    print(f"{status} ComfyUI: {result}")

# Check Backend API
backend_url = os.getenv("BACKEND_API_URL")
if backend_url:
    success, result = await check_http_service("Backend", backend_url, "/health")
    status = "✅" if success else "❌"
    print(f"{status} Backend API: {result}")

## 3. Database Connectivity

In [ ]:
print("\n💾 Database Connectivity:")
print("-" * 60)

# Check PostgreSQL
try:
    from sqlalchemy import create_engine
    db_url = os.getenv("DATABASE_URL")
    if db_url:
        engine = create_engine(db_url)
        with engine.connect() as conn:
            result = conn.execute("SELECT version()")
            version = result.fetchone()[0]
            print(f"✅ PostgreSQL: Connected ({version[:30]}...)")
    else:
        print("❌ PostgreSQL: DATABASE_URL not set")
except Exception as e:
    print(f"❌ PostgreSQL: {e}")

# Check Redis
try:
    import redis
    redis_url = os.getenv("REDIS_URL")
    if redis_url:
        r = redis.from_url(redis_url)
        r.ping()
        print(f"✅ Redis: Connected")
    else:
        print("❌ Redis: REDIS_URL not set")
except Exception as e:
    print(f"❌ Redis: {e}")

# Check Neo4j
try:
    from neo4j import GraphDatabase
    neo4j_uri = os.getenv("NEO4J_URI")
    neo4j_user = os.getenv("NEO4J_USER")
    neo4j_pass = os.getenv("NEO4J_PASSWORD")
    
    if neo4j_uri and neo4j_user and neo4j_pass:
        driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_pass))
        driver.verify_connectivity()
        print(f"✅ Neo4j: Connected")
        driver.close()
    else:
        print("❌ Neo4j: Connection details not set")
except Exception as e:
    print(f"❌ Neo4j: {e}")

## 4. Summary

All checks complete! If any services show as ❌, verify:
1. Service is enabled in `.env` (not set to `disabled`)
2. Service is running: `docker compose ps`
3. Environment variables are correctly set

## Next Steps

- `01_ollama_basics.ipynb` - Learn how to use Ollama for LLM inference
- `02_langchain_rag.ipynb` - Build a RAG pipeline with Weaviate
- `03_neo4j_graphs.ipynb` - Work with knowledge graphs
- `04_supabase_data.ipynb` - Database and storage operations
- `05_comfyui_images.ipynb` - Generate images with AI
- `06_n8n_workflows.ipynb` - Automate workflows